In [1]:
import pandas as pd
from collections import Counter

In [2]:
# Función para detectar ciclos de movimientos consecutivos.
def detect_cycles(group, n=3):
    movements = group["MOVEMENT"].values
    agents = group["AGENT.ID"].values

    # Inicializar contador de pares de agentes.
    pair_counts = Counter()

    # Generar patrones alternos de entrada y salida.
    for i in range(len(movements) - n + 1):
        # Verificar si la secuencia alterna (in, out, in, ...) o (out, in, out, ...) existe.
        in_out_pattern = all(movements[i+j] == ("in" if j % 2 == 0 else "out") for j in range(n))
        out_in_pattern = all(movements[i+j] == ("out" if j % 2 == 0 else "in") for j in range(n))


        if in_out_pattern or out_in_pattern:
            # Contar los pares únicos de agentes involucrados.
            cycle_agents = agents[i:i+n]
            unique_agents = sorted(set(cycle_agents))  # Eliminar duplicados y ordenar.
            
            # Generar los pares de agentes.
            for i in range(len(unique_agents)):
                for j in range(i + 1, len(unique_agents)):
                    pair_counts[(unique_agents[i], unique_agents[j])] += 1

    return pair_counts

In [3]:
# Cargar el archivo CSV.
data = pd.read_csv("data/ClientMovementsSN.csv", delimiter=";")

# Convertir la columna de fecha al formato adecuado.
data['DATE'] = pd.to_datetime(data['DATE'], format="%d/%m/%Y")

# Filtrar las filas donde el competidor 10 está involucrado.
filtered_data = data[
    (data['TRANSFEROR'] == "Competitor 10") | (data['RECEPIENT'] == "Competitor 10")
]

# Ordenar los datos.
filtered_data = filtered_data.sort_values(['ClientID', 'DATE'])

display(filtered_data)

,DATE,ClientID,MOVEMENT,BALANCE_IN,TRANSFEROR,BALANCE_OUT,RECEPIENT,AGENT.ID,STATUS,CHANNEL,aux,IN,OUT
1,2018-02-28,2,out,0,Competitor 9,0,Competitor 10,13087,Active,CHANNEL 5,1,0,1
2,2017-01-31,3,out,0,Competitor 9,17552962,Competitor 10,18024,Active,CHANNEL 5,1,0,1
4,2017-03-31,5,out,0,Competitor 9,29433839,Competitor 10,30677,Active,CHANNEL 5,1,0,1
5,2016-09-30,6,out,0,Competitor 9,6352446,Competitor 10,19040,Active,CHANNEL 5,1,0,1
6,2018-05-31,7,out,0,Competitor 9,11948542,Competitor 10,14204,Active,CHANNEL 5,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
749187,2020-01-31,689739,out,0,Competitor 9,18010023,Competitor 10,14586,Active,CHANNEL 5,1,0,1
749189,2019-03-31,689740,out,0,Competitor 9,1611736,Competitor 10,23885,Active,CHANNEL 5,1,0,1
749190,2017-09-30,689741,out,0,Competitor 9,565283,Competitor 10,5036,Active,CHANNEL 5,1,0,1
749194,2018-10-31,689745,out,0,Competitor 9,13770483,Competitor 10,16600,Active,CHANNEL 5,1,0,1


In [4]:
# Inicializar un contador global para los pares de agentes.
global_pair_counts = Counter()

# Listado para almacenar los resultados finales.
all_pairs = []

# Iterar sobre los datos agrupados por "ClientID".
for client_id, group in filtered_data.groupby("ClientID"):
    n = 2  # Iniciar en 2.
    while True:
        # Detectar ciclos con el valor de n.
        pair_counts = detect_cycles(group, n=n)

        # Salir si no se encontraron ciclos.
        if not pair_counts:
            break

        # Actualizar el contador global con los pares encontrados.
        global_pair_counts.update(pair_counts)

        # Almacenar los resultados con la intensidad (n).
        for pair, count in pair_counts.items():
            all_pairs.append({"AgentPair": pair, "ClientCount": count, "Intensity": n})

        # Incrementar n para la siguiente iteración.
        n += 1

# Convertir el listado de resultados en un DataFrame.
pairs_df = pd.DataFrame(all_pairs)

# Separar los pares en columnas "Agent1" y "Agent2".
pairs_df[["Agent1", "Agent2"]] = pd.DataFrame(pairs_df["AgentPair"].tolist(), index=pairs_df.index)

# Eliminar la columna redundante de tuplas.
pairs_df = pairs_df.drop(columns=["AgentPair"])

# Ordenar por número de clientes compartidos.
pairs_df = pairs_df.sort_values(by="ClientCount", ascending=False).reset_index(drop=True)

# Mostrar el resultado final.
display(pairs_df)

,ClientCount,Intensity,Agent1,Agent2
0,2,3,87,2553
1,2,4,87,2553
2,2,4,87,2982
3,2,4,2553,2982
4,2,3,87,2982
...,...,...,...,...
307,1,2,11348,19643
308,1,3,19921,24184
309,1,2,19921,24184
310,1,2,3556,16315


In [5]:
# Calcular estadísticas de los agentes.
agent_stats = (
    data.groupby("AGENT.ID")
    .agg(
        TotalIn=("BALANCE_IN", "sum"),
        TotalOut=("BALANCE_OUT", "sum"),
        Movements=("ClientID", "count"),
    )
    .reset_index()
)
agent_stats["NetBalance"] = agent_stats["TotalIn"] - agent_stats["TotalOut"]

# Mostrar las estadísticas de los agentes.
display(agent_stats)

,AGENT.ID,TotalIn,TotalOut,Movements,NetBalance
0,1,0,230657983,13,-230657983
1,2,2064711094,947219471,147,1117491623
2,3,0,2458069,1,-2458069
3,4,0,265623061,11,-265623061
4,5,0,61070458,24,-61070458
...,...,...,...,...,...
31286,31287,0,39392509,2,-39392509
31287,31288,0,13108566,1,-13108566
31288,31289,0,91244044,17,-91244044
31289,31290,0,668494,1,-668494


In [20]:
pairs_df.to_csv('/home/andres/Público/pairs_df.csv', index=False)
agent_stats.to_csv('/home/andres/Público/agent_stats.csv', index=False)

In [7]:
import networkx as nx

G = nx.Graph()
for _, row in pairs_df.iterrows():
    G.add_edge(row["Agent1"], row["Agent2"], weight=row["ClientCount"])

closeness = nx.closeness_centrality(G)
degree = dict(G.degree(weight="weight"))

In [11]:
# Convertir el diccionario de closeness a un DataFrame
closeness_df = pd.DataFrame(list(closeness.items()), columns=['Agent', 'Closeness'])

# Guardar el DataFrame en un archivo CSV
closeness_df.to_csv(r'C:\Users\seelro06\OneDrive - Arca Continental S.A.B. de C.V\Documentos\uni\del-negociante\data\closeness_centrality.csv', index=False)

display(closeness_df)

,Agent,Closeness
0,87,0.012461
1,2553,0.012461
2,2982,0.012461
3,8410,0.021807
4,30621,0.024424
...,...,...
317,19643,0.004984
318,3556,0.003115
319,16315,0.003115
320,5692,0.003115


In [9]:
display(degree)

{87: 4,
 2553: 4,
 2982: 4,
 8410: 6,
 30621: 6,
 1240: 4,
 5225: 4,
 7552: 2,
 18315: 2,
 1654: 2,
 10600: 1,
 30282: 1,
 14314: 1,
 27522: 2,
 19008: 3,
 26799: 2,
 2476: 1,
 19238: 1,
 16600: 1,
 22543: 3,
 4944: 1,
 24523: 1,
 5293: 1,
 10475: 1,
 8250: 2,
 18487: 1,
 22205: 2,
 30495: 1,
 2214: 1,
 19601: 1,
 14856: 1,
 24492: 5,
 20508: 1,
 29088: 1,
 6662: 2,
 7988: 2,
 17135: 1,
 22140: 1,
 3056: 2,
 4647: 2,
 3155: 2,
 3122: 2,
 30630: 2,
 1044: 2,
 4460: 2,
 6418: 2,
 3422: 2,
 4215: 2,
 4664: 2,
 3351: 2,
 13737: 2,
 28573: 2,
 1388: 2,
 6255: 2,
 6499: 2,
 2041: 2,
 8677: 2,
 29813: 2,
 3421: 2,
 4774: 2,
 5297: 2,
 1396: 2,
 3994: 2,
 31139: 2,
 3413: 2,
 9763: 2,
 12010: 2,
 3758: 2,
 13010: 1,
 25457: 1,
 30351: 2,
 3058: 1,
 14971: 6,
 20953: 1,
 17718: 1,
 28097: 1,
 837: 1,
 4401: 1,
 17822: 1,
 18700: 1,
 3206: 2,
 25491: 3,
 5693: 2,
 25618: 1,
 16963: 1,
 22177: 1,
 12751: 1,
 18065: 1,
 23944: 3,
 3083: 1,
 22190: 2,
 6415: 1,
 30897: 1,
 281: 1,
 3453: 1,
 11348: